In [ ]:
import pandas_datareader.data as web
import pandas as pd

# for financial stocks prediction simulation
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import random 
import math
style.use('ggplot')

# for casino : The house always wins! simulation
from tqdm import tqdm, trange 
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.figure_factory as ff
pyo.init_notebook_mode(connected= True)

# Sic Bo Game : Simulating that the House always wins!

In [ ]:
# 1 game
def game(money,bet):
    r = np.random.randint(1,1000)

    # if 1-486, means we picked small / big , we win
    # else we lost
    if r <= 486:
        money = money + bet
    else:
        money = money - bet

    return money

# 1 try (1 instance)
def instance_in_casino(money,bet,n_games):
    # store money
    money_df = pd.DataFrame(columns = {'money','game'})
    current_money = np.copy(money)
    for i in range(1,n_games+1):
        game_df = pd.DataFrame(columns = {'money','game'})
        current_money = game(current_money,bet)
        game_df['money'] = [current_money]
        game_df['game'] = [i]

        # append money
        money_df = pd.concat([money_df,game_df], axis = 0)
    return money_df
# n tries (simulation) n instances 
# store all instances in one df 

def multiple_instance(money,bet,n_games,n_instance):
    output = pd.DataFrame()
    # loop instances 
    for j in trange(1,n_instance+1):
        temp = instance_in_casino(money,bet,n_games)
        temp['instance'] = j
        output = pd.concat([output,temp], axis = 0)
        del temp    
    return output 

def plot_monte_carlo_results(df):
    # append each game's result into data for plot 
    data = []
    n_unique = len(df['instance'].unique())
    for k in trange(1,n_unique+1):
        dff = df[df['instance'] == k]
        t = go.Scatter(x = dff['game'], y = dff['money'], mode = 'lines',hoverinfo='none')
        data.append(t)
    return data

# define some layout for our plots 
def plot_layout(n_games,mean_value):
    
    layout = go.Layout(title = 'Monte Carlo Simulation of Simplified Small & Big (Sic Bo)',
                    shapes = [dict(type = 'line',xref = 'x', yref = 'y',x0 = 0,y0 = mean_value, x1 = n_games, y1 = mean_value,line=dict(
                    color="red",
                    width=5))],
                      xaxis = dict(title = 'Number of games',dtick = 1),
                      yaxis = dict(title = 'Money ($)'),
                      showlegend = False,
                      template = 'plotly_dark',
                      annotations=[
                          dict(text="The red line is representing the Mean Value = {} $".format(mean_value),
                          showarrow=True )]) 
    return layout

# expected money 
def expected_money(df,n_games,n_instance):
    histogram_df = df[df['game'] == n_games]
    histogram_df.money = histogram_df.money.astype(np.int)   
    expected_reward = np.mean(histogram_df['money'])  
    return expected_reward

# 1000 instances of 10 games each

In [ ]:
test = multiple_instance(1000,100,10,1000)
test.head(20)

In [ ]:
data = plot_monte_carlo_results(test)

In [ ]:
expected_money(test,10,1000)

In [ ]:
layout = plot_layout(10,expected_money(test,10,1000))
pyo.iplot(go.Figure(data, layout),config={'displayModeBar': True})

# 5000 instances of 10 games each

In [ ]:
five_thousand_instance = multiple_instance(1000,100,10,5000)

In [ ]:
data2 = plot_monte_carlo_results(five_thousand_instance)

In [ ]:
expected_money(five_thousand_instance,10,5000)

In [ ]:
layout = plot_layout(10,expected_money(five_thousand_instance,10,5000))

In [ ]:
pyo.iplot(go.Figure(data2, layout))

# 2000 instances with 20 games

In [ ]:
two_thousand_instances = multiple_instance(1000,100,20,2000)

In [ ]:
data3 = plot_monte_carlo_results(two_thousand_instances)

In [ ]:
expected_money(two_thousand_instances,20,2000)

In [ ]:
layout = plot_layout(20,expected_money(two_thousand_instances,20,2000))

In [ ]:
pyo.iplot(go.Figure(data3, layout))

# Definite Integration :

*   Finding an approximate solution to a definate intergal problem
*   Assumptions : We know the limits 



In [ ]:
# using monte carlo methods to approximately solve an integral of log(x) / x from 1 to 10 limits:

def function(x):
  return math.log(x) / x

count = 0.0
in_area = 0.0

while count < 10000:   # our sampling quantity
  x_coord = random.uniform(1,10)
  y_coord = random.uniform(0,1/math.exp(1))

  if y_coord < function(x_coord):
    in_area+=1

  count+=1
area_box = 9 / math.exp(1)

print((in_area / count) * area_box) 
absolute_error = 1 / math.sqrt(count)
print(absolute_error)

# Estimating value of π : 

In [ ]:
Interval= 100
circle_points = 0
square_points = 0
for i in range(Interval**2):
# Range of x and y values is -1 to 1
    x= random.uniform(-1, 1)
    y= random.uniform(-1, 1)
# Distance between (x, y) from the origin
    origin_dist= x**2 + y**2

# Checking if (x, y) lies inside the circle
    if origin_dist<= 1:
        circle_points+= 1

    square_points+= 1

# Estimating value of pi,formula we have used : 
# pi = 4 * (no. of points generated inside the circle)/ (no. of points generated inside the square)
    pi = 4* circle_points/ square_points
#     print(x, y, circle_points, square_points, "-", pi)
#     print("\n\n")
print("Final Estimation of Pi=", pi)

# Prelimnary Stocks Price Prediction Simulation

In [ ]:
start = dt.datetime(2020,1,1)
end = dt.datetime(2020,12,1)

prices = web.DataReader('AAPL','iex',start,end,api_key='sk_e8941bb4d5b048c4bcbc13ebc1b0f32a')['close']

In [ ]:
prices.describe()
returns = prices.pct_change()
last_price = prices[-1]
# time_diff = (dt.date(2020,1,1) - dt.date(2020,12,1)).days
# print(time_diff)

In [ ]:
# number of trials :

num_simulations = 1000
num_days = 251

simulation_df = pd.DataFrame()

for x in range(num_simulations):
  count = 0
  daily_vol = returns.std()
  price_series = []
  price_series = []
    
  price = last_price * (1 + np.random.normal(0, daily_vol))
  price_series.append(price)
    
  for y in range(num_days):
    if (count == 232):
        break
    price = price_series[count] * (1 + np.random.normal(0, daily_vol))
    price_series.append(price)
    count += 1
    
  simulation_df[x] = price_series
fig = plt.figure(figsize = (15,8),facecolor = 'black')
fig.suptitle('Monte Carlo Simulation: AAPL',color = 'white',fontsize = 20)
plt.plot(simulation_df)
plt.axhline(y = last_price, color = 'r', linestyle = '-',label = last_price)
plt.xlabel('Day',color = 'white',fontsize = 15)
plt.tick_params(axis='x', colors='white')
plt.tick_params(axis='y', colors='white')
plt.text(-5.5, 300.5, 'last price : {}'.format(last_price), bbox=dict(facecolor='yellow', alpha=0.5))
plt.ylabel('Price',color = 'white',fontsize = 15)
plt.show()
